PPO vs TD3

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import torch
import torch.nn as nn
import gym
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

# Adaptive Normalization
window_size = 126  
rolling_means = returns.rolling(window=window_size, min_periods=1).mean()
rolling_stds = returns.rolling(window=window_size, min_periods=1).std()
rolling_stds.replace(0, np.nan, inplace=True)
adaptive_returns = (returns - rolling_means) / rolling_stds
adaptive_returns.dropna(inplace=True)

# Ensure 'returns' and 'adaptive_returns' have matching indexes
aligned_returns = returns.loc[adaptive_returns.index].copy()

# Apply PCA to reduced adaptive returns
pca = PCA(n_components=4)
reduced_features = pca.fit_transform(adaptive_returns)

# K-Means Clustering
n_clusters = 4  
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
aligned_returns['Cluster'] = kmeans.fit_predict(reduced_features)

# One-Hot Encode Clusters
ohe = OneHotEncoder(sparse=False)
cluster_features = ohe.fit_transform(aligned_returns[['Cluster']])
cluster_features_df = pd.DataFrame(cluster_features, columns=[f'Cluster_{i}' for i in range(n_clusters)], index=aligned_returns.index)

# Merge into final dataset
returns = pd.concat([aligned_returns.drop(columns=['Cluster']), cluster_features_df], axis=1)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
import gym
from stable_baselines3 import PPO, TD3
from stable_baselines3.common.vec_env import DummyVecEnv

# Custom Portfolio Environment
class PortfolioEnv(gym.Env):
    def __init__(self, returns):
        super(PortfolioEnv, self).__init__()
        self.returns = returns
        self.n_assets = returns.shape[1] - (n_clusters + 1)  # Excluding cluster labels and one-hot features
        self.observation_dim = self.n_assets + n_clusters  # Ensure correct shape
        self.action_space = gym.spaces.Box(low=0, high=1, shape=(self.n_assets,), dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(self.observation_dim,), dtype=np.float32)
        self.current_step = 0
    
    def reset(self):
        self.current_step = 0
        obs = np.append(
            self.returns.iloc[self.current_step, :-n_clusters-1].values,
            self.returns.iloc[self.current_step, -n_clusters:].values
        )
        return obs[:self.observation_dim]  
    
    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.returns) - 1
        
        portfolio_return = np.dot(action, self.returns.iloc[self.current_step, :-n_clusters-1].values)
        volatility = np.std(np.dot(self.returns.iloc[:self.current_step+1, :-n_clusters-1], action))
        var_threshold = np.percentile(portfolio_return, 5)
        cvar = np.mean(portfolio_return[portfolio_return <= var_threshold])
        r2r = portfolio_return / (volatility + 1e-6)
        
        reward =  abs(cvar) + r2r
        obs = np.append(
            self.returns.iloc[self.current_step, :-n_clusters-1].values,
            self.returns.iloc[self.current_step, -n_clusters:].values
        )
        
        return obs[:self.observation_dim], reward, done, {"r2r": r2r, "cvar": cvar, "portfolio_return": portfolio_return}
    
# Create Environment
env = DummyVecEnv([lambda: PortfolioEnv(returns)])

# Train PPO Model
ppo_model = PPO("MlpPolicy", env, verbose=1)
ppo_model.learn(total_timesteps=100000)

# Train TD3 Model
td3_model = TD3("MlpPolicy", env, verbose=1)
td3_model.learn(total_timesteps=100000)

# Evaluate Performance with clipped rewards
def evaluate_model(model, env, print_interval=100, reward_clip_range=(-1, 1)):
    obs = env.reset()
    done, rewards, cumulative_returns = False, [], []
    total_return = 1
    metrics = {"r2r": [], "cvar": [], "portfolio_return": []}
    
    step_counter = 0
    
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        
        # Clip rewards to avoid overflow
        reward = np.clip(reward, reward_clip_range[0], reward_clip_range[1])
        
        rewards.append(reward)
        total_return *= (1 + reward)
        
        # Prevent total_return from reaching infinity
        if np.isnan(total_return) or np.isinf(total_return):
            total_return = np.clip(total_return, -1e10, 1e10)
        
        cumulative_returns.append(total_return)
        
        info = info[0]  
        metrics["r2r"].append(info["r2r"])
        metrics["cvar"].append(info["cvar"])
        metrics["portfolio_return"].append(info["portfolio_return"])

        #step_counter += 1
        
    return np.mean(rewards), cumulative_returns, metrics

ppo_reward, ppo_cum_returns, ppo_metrics = evaluate_model(ppo_model, env)
td3_reward, td3_cum_returns, td3_metrics = evaluate_model(td3_model, env)

print(f"PPO Metrics - R2R: {np.mean(ppo_metrics['r2r'])}, CVaR: {np.mean(ppo_metrics['cvar'])}")
print(f"TD3 Metrics -  R2R: {np.mean(td3_metrics['r2r'])}, CVaR: {np.mean(td3_metrics['cvar'])}")


In [ ]:
# Convert portfolio returns to Pandas Series
ppo_portfolio_return = pd.Series(ppo_metrics["portfolio_return"])
td3_portfolio_return = pd.Series(td3_metrics["portfolio_return"])

# Compute rolling volatility (standard deviation over a 6-month window)
ppo_rolling_vol = ppo_portfolio_return.rolling(window=126).std()
td3_rolling_vol = td3_portfolio_return.rolling(window=126).std()

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(dates, ppo_rolling_vol, label="PPO Rolling Volatility", color="blue")
plt.plot(dates, td3_rolling_vol, label="TD3 Rolling Volatility", color="red")

plt.xlabel("Date")
plt.ylabel("Volatility (Rolling 6 Months)")
plt.title("Rolling Volatility Over Time (6-Month Window)")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.show()


# Summarize portfolio return contributions
ppo_total_return = sum(ppo_metrics["portfolio_return"])
td3_total_return = sum(td3_metrics["portfolio_return"])

# Pie chart labels
labels = ["PPO Portfolio Return", "TD3 Portfolio Return"]
returns = [ppo_total_return, td3_total_return]

# Define colors
colors = ["blue", "red"]

# Create Pie Chart
plt.figure(figsize=(8, 6))
plt.pie(returns, labels=labels, autopct="%1.1f%%", colors=colors, startangle=140, explode=[0.05, 0])
plt.title("Portfolio Return Contribution (PPO vs TD3)")
plt.show()



PPO-LSTM

In [ ]:
# === Custom Portfolio Environment ===
class PortfolioEnv(gym.Env):
    def __init__(self, returns, n_clusters):
        super(PortfolioEnv, self).__init__()
        self.returns = returns
        self.n_assets = len(price_cols)
        self.n_clusters = n_clusters
        self.action_space = gym.spaces.Box(low=0, high=1, shape=(self.n_assets,), dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(self.n_assets + self.n_clusters,), dtype=np.float32)
        self.current_step = 0
        self.initial_capital = 1.0

    def reset(self):
        self.current_step = 0
        self.capital = self.initial_capital
        obs = np.append(self.returns.iloc[self.current_step, :-self.n_clusters].values, 
                        self.returns.iloc[self.current_step, -self.n_clusters:].values)
        obs = self._fix_observation_shape(obs)
        return np.nan_to_num(obs)

    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.returns) - 1

        action = np.nan_to_num(action)
        exp_a = np.exp(action - np.max(action))  
        action = exp_a / np.sum(exp_a)

        portfolio_return = np.dot(action, self.returns.iloc[self.current_step, :-self.n_clusters].values)
        self.capital *= (1 + portfolio_return)

        rolling_window = min(126, self.current_step + 1)
        volatility = np.std(np.dot(self.returns.iloc[self.current_step - rolling_window + 1:self.current_step + 1, :-self.n_clusters], action))
        volatility = max(volatility, 1e-6)

        past_returns = np.dot(self.returns.iloc[:self.current_step + 1, :-self.n_clusters], action)
        var_threshold = np.percentile(past_returns, 5)
        cvar = np.mean(past_returns[past_returns <= var_threshold]) if np.any(past_returns <= var_threshold) else 0

        alpha, beta = 0.3, 0.2
        r2r = portfolio_return / volatility
        drawdown_penalty = abs(min(0, portfolio_return))
        reward = r2r - alpha * abs(cvar) - beta * drawdown_penalty

        obs = np.append(self.returns.iloc[self.current_step, :-self.n_clusters].values,
                        self.returns.iloc[self.current_step, -self.n_clusters:].values)
        obs = self._fix_observation_shape(obs)
        return np.nan_to_num(obs), reward, done, {"capital": self.capital, "r2r": r2r, "cvar": cvar}

    def _fix_observation_shape(self, obs):
        expected_shape = self.n_assets + self.n_clusters
        if obs.shape[0] > expected_shape:
            obs = obs[:expected_shape]
        elif obs.shape[0] < expected_shape:
            obs = np.pad(obs, (0, expected_shape - obs.shape[0]), 'constant')
        return obs

# === LSTM Feature Extractor ===
class LSTMFeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=128):
        super(LSTMFeatureExtractor, self).__init__(observation_space, features_dim)
        input_dim = observation_space.shape[0]
        self.lstm = nn.LSTM(input_dim, features_dim, batch_first=True)
        self.flatten = nn.Flatten()

    def forward(self, observations):
        observations = observations.unsqueeze(1)
        lstm_out, _ = self.lstm(observations)
        lstm_out = self.flatten(lstm_out[:, -1, :])
        return lstm_out

# === Evaluation Function ===
def evaluate_model(model, env, threshold=0.0):  
    obs = env.reset()
    done, cumulative_returns = False, []
    r2r_values, cvar_values = [], []
    daily_returns = []

    while not done:
        action, _ = model.predict(np.nan_to_num(obs))
        obs, reward, done, info = env.step(action)
        cumulative_returns.append(info[0]["capital"])
        r2r_values.append(info[0]["r2r"])
        cvar_values.append(info[0]["cvar"])

        if len(cumulative_returns) > 1:
            ret = cumulative_returns[-1] / cumulative_returns[-2] - 1
            daily_returns.append(ret)

    peak = np.maximum.accumulate(cumulative_returns)
    drawdowns = (peak - cumulative_returns) / peak
    max_drawdown = np.max(drawdowns)

    gains = [r for r in daily_returns if r > threshold]
    losses = [abs(r) for r in daily_returns if r < threshold]
    omega_ratio = (np.sum(gains) / np.sum(losses)) if np.sum(losses) > 0 else np.inf

    return None, cumulative_returns, r2r_values, cvar_values, max_drawdown, omega_ratio

train_returns = returns.copy()
test_returns = returns.copy()

# === Multi-Seed Training ===
ppo_results = []
ppo_lstm_results = []
seeds = [0, 42, 123, 777, 999]

for seed in seeds:
    print(f"\n=== Training with Seed: {seed} ===")
    np.random.seed(seed)
    torch.manual_seed(seed)

    train_env = DummyVecEnv([lambda: PortfolioEnv(train_returns, n_clusters)])
    test_env = DummyVecEnv([lambda: PortfolioEnv(test_returns, n_clusters)])

    # PPO-LSTM
    ppo_lstm_model = PPO(
        "MlpPolicy", train_env,
        policy_kwargs={"features_extractor_class": LSTMFeatureExtractor},
        gamma=0.993, learning_rate=0.0002, batch_size=128, ent_coef=0.012, verbose=0, seed=seed
    )
    ppo_lstm_model.learn(total_timesteps=100000)

    _, cum_returns, r2r_vals, cvar_vals, max_dd, omega = evaluate_model(ppo_lstm_model, test_env)
    ppo_lstm_results.append({
        "final_return": cum_returns[-1],
        "r2r": np.mean(r2r_vals),
        "cvar": np.mean(cvar_vals),
        "max_dd": max_dd,
        "omega": omega,
        "capital_curve": cum_returns,
        "drawdown_curve": (np.maximum.accumulate(cum_returns) - cum_returns) / np.maximum.accumulate(cum_returns)
    })

    # PPO
    ppo_model = PPO(
        "MlpPolicy", train_env,
        gamma=0.91, learning_rate=0.0002, batch_size=64, ent_coef=0.05, verbose=0, seed=seed
    )
    ppo_model.learn(total_timesteps=100000)

    _, cum_returns, r2r_vals, cvar_vals, max_dd, omega = evaluate_model(ppo_model, test_env)
    ppo_results.append({
        "final_return": cum_returns[-1],
        "r2r": np.mean(r2r_vals),
        "cvar": np.mean(cvar_vals),
        "max_dd": max_dd,
        "omega": omega,
        "capital_curve": cum_returns,
        "drawdown_curve": (np.maximum.accumulate(cum_returns) - cum_returns) / np.maximum.accumulate(cum_returns)
    })

# === Summarize Results ===
def summarize_results(results, label):
    df = pd.DataFrame(results)
    print(f"\n{label} Results (mean ± std):")
    print(f"Final Return: {df['final_return'].mean():.4f} ± {df['final_return'].std():.4f}")
    print(f"R2R: {df['r2r'].mean():.4f} ± {df['r2r'].std():.4f}")
    print(f"CVaR: {df['cvar'].mean():.4f} ± {df['cvar'].std():.4f}")
    print(f"Max Drawdown: {df['max_dd'].mean():.4f} ± {df['max_dd'].std():.4f}")
    print(f"Omega: {df['omega'].mean():.4f} ± {df['omega'].std():.4f}")

summarize_results(ppo_lstm_results, "PPO-LSTM")
summarize_results(ppo_results, "PPO")


In [ ]:
import itertools
# Calculate average cumulative return across all seeds
ppo_lstm_final_returns = [result['final_return'] for result in ppo_lstm_results]
ppo_final_returns = [result['final_return'] for result in ppo_results]

# Convert to percentages
avg_lstm_return = np.mean(ppo_lstm_final_returns) * 100
avg_ppo_return = np.mean(ppo_final_returns) * 100

# Calculate performance improvement
improvement = avg_lstm_return - avg_ppo_return
relative_improvement = (improvement / avg_ppo_return) * 100

# Print results
print(f"PPO-LSTM Average Return: {avg_lstm_return:.2f}%")
print(f"PPO Average Return: {avg_ppo_return:.2f}%")
print(f"Absolute Improvement: {improvement:.2f}%")
print(f"Relative Improvement: {relative_improvement:.2f}%")


# Define distinct styles
ppo_colors = plt.cm.Blues(np.linspace(0.4, 1, len(seeds)))
ppo_lstm_colors = plt.cm.Reds(np.linspace(0.4, 1, len(seeds)))
line_styles = ['-', '--', '-.', ':', (0, (3, 5, 1, 5))]

plt.figure(figsize=(12, 6))

# Plot PPO results
for i, result in enumerate(ppo_results):
    dates_for_plot = data.index[-len(result['capital_curve']):]
    plt.plot(dates_for_plot,
             result['capital_curve'],
             color=ppo_colors[i],
             linestyle=line_styles[i % len(line_styles)],
             linewidth=2,
             label=f'PPO Seed {seeds[i]}')

# Plot PPO-LSTM results
for i, result in enumerate(ppo_lstm_results):
    dates_for_plot = data.index[-len(result['capital_curve']):]
    plt.plot(dates_for_plot,
             result['capital_curve'],
             color=ppo_lstm_colors[i],
             linestyle=line_styles[i % len(line_styles)],
             linewidth=2,
             label=f'PPO-LSTM Seed {seeds[i]}')

plt.title('Capital Growth Comparison (PPO vs PPO-LSTM)')
plt.xlabel('Date')
plt.ylabel('Capital')
plt.legend(loc='upper left', fontsize=9)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


# Define consistent colors
ppo_color = 'steelblue'
ppo_lstm_color = 'firebrick'

# Create the figure
plt.figure(figsize=(12, 6))

# PPO Drawdown (solid lines)
for i, result in enumerate(ppo_results):
    dates_for_plot = data.index[-len(result['drawdown_curve']):]
    plt.plot(dates_for_plot,
             result['drawdown_curve'],
             label=f'PPO Seed {seeds[i]}',
             color=ppo_color,
             linestyle='-',
             linewidth=1.8)

# PPO-LSTM Drawdown (dashed lines)
for i, result in enumerate(ppo_lstm_results):
    dates_for_plot = data.index[-len(result['drawdown_curve']):]
    plt.plot(dates_for_plot,
             result['drawdown_curve'],
             label=f'PPO-LSTM Seed {seeds[i]}',
             color=ppo_lstm_color,
             linestyle='--',
             linewidth=2.2)

# Final formatting
plt.title('Drawdown Comparison (PPO vs PPO-LSTM)', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Drawdown')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(fontsize=10, loc='upper left')
plt.tight_layout()
plt.show()


final_returns_ppolstm = [result['final_return'] for result in ppo_lstm_results]
final_returns_ppo = [result['final_return'] for result in ppo_results]
plt.figure(figsize=(8, 6))
plt.boxplot([final_returns_ppolstm, final_returns_ppo], labels=['PPO-LSTM', 'PPO'])
plt.title('Final Return Comparison (Boxplot)')
plt.ylabel('Final Return')
plt.grid(True)
plt.show()



window = 126  # ~6 months

capital_curve = np.array(ppo_lstm_results[0]['capital_curve'])
daily_returns = np.diff(capital_curve) / capital_curve[:-1]
returns_series = pd.Series(daily_returns)
rolling_vol = returns_series.rolling(window).std() * np.sqrt(252)
rolling_ret = pd.Series(capital_curve).pct_change(window)

vol_dates = data.index[-len(rolling_vol):]
ret_dates = data.index[-len(rolling_ret):]

fig, ax1 = plt.subplots(figsize=(12, 5))

ax1.set_xlabel('Date')
ax1.set_ylabel('Volatility', color='blue')
ax1.plot(vol_dates, rolling_vol, color='blue', label='Rolling Volatility')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
ax2.set_ylabel('Return', color='red')
ax2.plot(ret_dates, rolling_ret, color='red', linestyle='--', label='Rolling Return')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('6-Month Rolling Volatility and Return')
fig.tight_layout()
plt.grid(True)
plt.show()


